In [47]:
#####ノイズあり対応トピックモデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
#文書の設定
k = 15   #トピック数
d = 4000   #文書数
v  = 1000   #語彙数
w = np.random.poisson(np.random.gamma(65, 1/0.5, d), d)   #1文書あたりの単語数
a1 = 150   #トピックに関係のあるタグ数
a2 = 100   #トピックに関係のないタグ数
a = a1 + a2
x = np.random.poisson(40, d)
x[x < 2] = 3
f1 = np.sum(w)
f2 = np.sum(x)

#IDの設定
w_id = np.repeat(range(d), w)
a_id = np.repeat(range(d), x)

In [48]:
##パラメータの設定
#ディレクリ事前分布のパラメータを設定
alpha0 = np.repeat(0.15, k)   #文書のディレクリ事前分布のパラメータ
alpha1 = np.repeat(0.15, v)   #単語のディレクリ事前分布のパラメータ
alpha2 = np.hstack((np.repeat(0.2, a1), np.repeat(0.001, a2)))   #トピックに関係のあるタグのディレクリ事前分布のパラメータ
alpha3 = np.hstack((np.repeat(0.001, a1), np.repeat(1.25, a2)))   #トピックに関係のないタグのディリクレ事前分布のパラメータ
beta0 = betat0 = np.random.beta(5.0, 2.0, d)

In [49]:
##すべての単語が出現するまで生成を繰り返す
for rp in range(1000):
    print(rp)
    
    #ディリクレ事前分布からパラメータを生成
    theta = thetat = np.random.dirichlet(alpha0, d)   #文書ごとのトピック分布
    phi = phit = np.random.dirichlet(alpha1, k)   #トピックごとの単語分布
    omega = omegat = np.random.dirichlet(alpha2, k)   #トピックに関係のある補助情報分布
    gamma = gammat = np.random.dirichlet(alpha3, 1)   #トピックに関係のない補助情報分布
    omega0 = np.vstack((omega, gamma))   #補助情報分布の結合
    
    #データの格納用配列
    WX = np.zeros((d, v))
    AX = np.zeros((d, a))
    aux_data = np.zeros((f2, a))
    word_list = [i for i in range(d)]
    aux_list = [i for i in range(d)]
    Z0 = np.repeat(0, f2)
    Z1_list = [i for i in range(d)]
    Z2_list = [i for i in range(d)]
    vec = np.arange(0, k)

    ##文書ごとにトピックと単語を生成する
    for i in range(d):
        
        #文書のトピックを生成
        z1 = np.random.multinomial(1, theta[i, :], w[i])
        z1_vec = np.dot(z1, vec)

        #文書のトピック分布から単語を生成
        word = np.zeros((w[i], v))
        for j in range(w[i]):
            word[j, :] = multinomial(1, phi[z1_vec[j], :], 1)
        WX[i, :] = np.sum(word, axis=0)


        #ベルヌーイ分布からトピックに関係があるかどうかを生成
        index = np.arange(f2)[a_id==i]
        Z0[index] = np.random.binomial(1, beta0[i], x[i])

        #文書のトピック分布から補助情報トピックを生成
        z2_aux = np.random.multinomial(1, theta[i, :], x[i])
        z2 = np.concatenate((z2_aux * Z0[index].reshape((x[i], 1)),1-Z0[index].reshape((x[i], 1))), axis=1)
        z2_vec = np.dot(z2, np.hstack((vec, k)))

        #生成したトピックの単語分布に従い単語を生成
        aux = np.zeros((x[i], a))
        for j in range(x[i]):
            aux[j, :] = np.random.multinomial(1, omega0[z2_vec[j], :], 1)
        AX[i, :] = np.sum(aux, axis=0)

        #文書トピックおよび補助情報トピックを格納
        Z1_list[i] = z1_vec
        Z2_list[i] = z2_vec
        word_list[i] = np.dot(word, np.arange(v))
        aux_list[i] = np.dot(aux, np.arange(a))
        aux_data[a_id==i, :] = aux
        
    if (np.min(np.sum(WX, axis=0)) > 0.0) & (np.min(np.sum(AX, axis=0)) > 0.0):
        break

0


In [50]:
#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ad = np.zeros(sum(x), dtype='int')
start1 = start2 = 0

for i in range(d):
    wd[start1:start1+w[i]] = word_list[i]
    ad[start2:start2+x[i]] = aux_list[i]
    start1 += w[i] 
    start2 += x[i]

In [51]:
##インデックスを作成
doc1_list = [i for i in range(d)]
doc1_vec = [i for i in range(d)]
doc2_list = [i for i in range(d)]
doc2_vec  = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
aux_list = [i for i in range(a)]
aux_vec = [i for i in range(a)]
index_doc = np.array(range(f1))
index_aux = np.array(range(f2))

for i in range(d):
    doc1_list[i] = index_doc[w_id==i]
    doc1_vec[i] = np.repeat(1, doc1_list[i].shape)
    doc2_list[i] = index_aux[a_id==i]
    doc2_vec[i] = np.repeat(1, doc2_list[i].shape)
for i in range(v):
    word_list[i] = index_doc[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)
for i in range(a):
    aux_list[i] = index_aux[ad==i]
    aux_vec[i] = np.repeat(1, aux_list[i].shape)

In [52]:
####ベイズ推定のための設定####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w_id, k):
    #尤度を計算
    Bur = theta[w_id, :]  * phi.T[wd, ]   #トピック尤度

    #負担率を計算
    Br = Bur / np.sum(Bur, axis=1).reshape((f, 1))   #負担率
    r = np.sum(Br, axis=0) / np.sum(Br)
    return Br, Bur, r


##アルゴリズムの設定
#更新ステータス
LLo = -100000000
LLw = LLo
iter = 1
dl = 100   #EMステップでの対数尤度の差の初期値
tol = 0.5

##事前分布の設定
alpha01 = 1
alpha02 = 1
beta01 = 0.25
beta02 = 0.25
gamma01 = 0.1
gamma02 = 0.1

##パラメータの初期値
#tf-idfで初期値を設定
tf = AX / np.sum(AX, axis=1).reshape((d, 1))
idf1 = np.log(AX.shape[0] / np.sum(AX > 0, axis=0))
idf2 = np.log(AX.shape[0] / np.sum(AX==0, axis=0))

##パラメータの真値
theta = thetat 
phi = phit
omega = omegat
gamma = gammat.reshape((a))
r = betat0

theta = np.random.dirichlet(np.repeat(1, k), d)   #文書トピックのパラメータの初期値
phi = np.random.dirichlet(np.repeat(2.0, v), k)   #単語分布のパラメータの初期値
omega = np.random.dirichlet(np.repeat(10, a), k)   #内容に関係のあるタグ分布のパラメータの初期値
gamma = np.random.dirichlet(np.repeat(10, a), 1).reshape((a))   #内容に関係のないタグ分布のパラメータの初期値
r = np.repeat(0.5, d)   #内容に関係があるかどうかの混合率

##対数尤度の基準値
LLst1 = np.sum(np.log((np.sum(WX, axis=0) / f1)[wd]))
LLst2 = np.sum(np.log((np.sum(AX, axis=0) / f2)[ad]))
LLst = LLst1 + LLst2
print(np.round(np.array((LLst, LLst1, LLst2)), 1))

[-4330845.5 -3482035.   -848810.5]


In [53]:
####ノイズあり対応トピックモデルをベイズ推定####
while abs(dl) >= tol: 
    
    ##単語トピックの期待値を推定
    #単語ごとにトピックの出現確率を計算
    word_par = burden_fr(f1, theta, phi, wd, w_id, k)
    Zi1 = word_par[0]
    Zi1_T = Zi1.T

    ##単語トピックのパラメータを更新
    #ディリクレ分布からthetaを更新
    wsum0 = np.zeros((d, k))
    for i in range(d):
        wsum0[i, :] = np.dot(Zi1_T[:, doc1_list[i]], doc1_vec[i])
    wsum = wsum0 + alpha01   #ディリクレ分布のパラメータ
    theta = wsum / np.sum(wsum, axis=1).reshape((d, 1))   #ディリクレ分布の期待値

    #ディリクレ分布からphiを更新
    vsum0 = np.zeros((k, v))
    for j in range(v):
        vsum0[:, j] = np.dot(Zi1_T[:, word_list[j]], word_vec[j])
    vsum = vsum0 + beta01   #ディリクレ分布のパラメタータ
    phi = vsum / np.sum(vsum, axis=1).reshape((k, 1))   #ディリクレ分布の期待値


    ##タグがトピックに関係があるかどうかの期待値
    #ベルヌーイ分布の確率を設定
    r_vec = r[a_id]   #混合率のベクトル
    tau01 = r_vec * np.sum(theta[a_id, :] * omega.T[ad, :], axis=1)   #補助変数トピックの尤度
    tau02 = (1-r_vec) * gamma[ad]   #ノイズ補助変数の尤度
    tau1 = tau01 / (tau01 + tau02)   #ベルヌーイ分布の期待値
    tau0 = 1-tau1

    #ベータ分布より混合率の期待値を更新
    for i in range(d):
        beta1 = np.sum(tau1[doc2_list[i]]) + gamma01
        beta2 = np.sum(tau0[doc2_list[i]]) + gamma02
        r[i] = beta1 / (beta1 + beta2)

    ##補助情報トピックの期待値を推定
    #タグごとにトピックの出現確率を計算
    aux_par = theta[a_id, :] * omega.T[ad, :]
    Zi2 = aux_par / np.sum(aux_par, axis=1).reshape((f2, 1))
    Zi2_T = Zi2.T

    ##補助情報トピックのパラメータを更新
    #ディリクレ分布からomegaを更新
    af0 = np.zeros((k, a))
    for j in range(a):
        af0[:, j] = np.dot((Zi2_T[:, aux_list[j]] * np.matlib.repmat((tau1[aux_list[j]]), k, 1)), aux_vec[j])
    af = af0 + gamma01   #ディリクレ分布のパラメータ
    omega = af / np.sum(af, axis=1).reshape((k, 1))   #ディリクレ分布の期待値

    #内容に関係のない補助情報分布omegaを更新
    nf = np.sum(tau0.reshape((f2, 1)) * aux_data, axis=0) + gamma02   #ディリクレ分布のパラメータ
    gamma = nf / np.sum(nf)   #ディリクレ分布の期待値


    #対数尤度の更新
    LL1 = np.sum(np.log(np.sum(word_par[1], axis=1)))
    LL2 = np.sum(np.log(tau0*gamma[ad] + tau1*np.sum(aux_par, axis=1)))
    LLs = LL1 + LL2   #対数尤度の総和
    iter = iter + 1
    dl = LLs-LLo
    LLo = LLs
    LLw = np.array((LLw, LLo))
    print(np.round(np.array((LLs, LLst)), 1))

[-4469655.1 -4330845.5]
[-4328551.4 -4330845.5]
[-4324998.2 -4330845.5]
[-4322076.6 -4330845.5]
[-4319170.6 -4330845.5]
[-4315862.7 -4330845.5]
[-4311766.2 -4330845.5]
[-4306441.9 -4330845.5]
[-4299350.6 -4330845.5]
[-4289866.6 -4330845.5]
[-4277423.5 -4330845.5]
[-4261792.2 -4330845.5]
[-4243342.6 -4330845.5]
[-4223128.8 -4330845.5]
[-4202644.2 -4330845.5]
[-4183295.5 -4330845.5]
[-4165948.4 -4330845.5]
[-4150835.5 -4330845.5]
[-4137766.4 -4330845.5]
[-4126372.9 -4330845.5]
[-4116247.8 -4330845.5]
[-4107016.8 -4330845.5]
[-4098409.1 -4330845.5]
[-4090325.6 -4330845.5]
[-4082830.4 -4330845.5]
[-4076029.9 -4330845.5]
[-4069940.5 -4330845.5]
[-4064448.5 -4330845.5]
[-4059342.4 -4330845.5]
[-4054381.1 -4330845.5]
[-4049394.7 -4330845.5]
[-4044392.  -4330845.5]
[-4039627.9 -4330845.5]
[-4035525.4 -4330845.5]
[-4032398.7 -4330845.5]
[-4030242.  -4330845.5]
[-4028821.5 -4330845.5]
[-4027879.9 -4330845.5]
[-4027231.9 -4330845.5]
[-4026761.9 -4330845.5]
[-4026401.  -4330845.5]
[-4026110.7 -433

[-3999921.5 -4330845.5]
[-3999920.9 -4330845.5]
[-3999920.3 -4330845.5]
[-3999919.7 -4330845.5]
[-3999919.1 -4330845.5]
[-3999918.6 -4330845.5]
[-3999918.  -4330845.5]
[-3999917.5 -4330845.5]
[-3999916.9 -4330845.5]
[-3999916.4 -4330845.5]
[-3999915.9 -4330845.5]
[-3999915.4 -4330845.5]
[-3999914.9 -4330845.5]


[-2790030.726 -3060060.841]


In [473]:
np.sum(np.log(np.sum(word_par[1], axis=1)))

-2421690.230698016

-2467888.2055135402

-2117331.6606235574

array([  5.48020320e-003,   1.51705730e-002,   1.34843855e-152, ...,
         1.07810786e-001,   4.51298023e-002,   1.83033615e-002])